In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from malaya_speech.torch_model.vits.model_infer import SynthesizerTrn
from malaya_boilerplate.train.config import HParams
import json

In [3]:
config = '/home/ubuntu/malay_vits/female_singlish.json'

In [4]:
with open(config) as fopen:
    hps = HParams(**json.load(fopen))

In [5]:
from malaya_speech.utils.text import TTS_SYMBOLS
import torch

In [6]:
model = SynthesizerTrn(len(TTS_SYMBOLS),
                                   hps.data.filter_length // 2 + 1,
                                   hps.train.segment_size // hps.data.hop_length,
                                   **hps.model)

In [7]:
_ = model.eval()

In [8]:
!ls /home/ubuntu/malay_vits/logs/female_singlish/G_*.pth

/home/ubuntu/malay_vits/logs/female_singlish/G_0.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_100000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_10000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_1000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_101000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_102000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_103000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_104000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_105000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_106000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_107000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_108000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_109000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_110000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_11000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_111000.pth
/home/ubuntu/malay_vits/logs/female_singlish/G_112000.pth
/home/ubuntu/malay_vits

In [9]:
pth = '/home/ubuntu/malay_vits/logs/female_singlish/G_119000.pth'

In [10]:
model.load_state_dict(torch.load(pth, map_location=torch.device('cpu'))['model'])

<All keys matched successfully>

In [11]:
from malaya_speech.torch_model.vits.commons import intersperse
import malaya_speech

In [12]:
normalizer = malaya_speech.utils.text.TextIDS(pad_to = None)

In [13]:
text = "GEORGE TOWN: A heritage activist has questioned the Penang government's continued silence on whether it would severely punish a land owner for illegally demolishing the 138-year-old grave of Foo Teng Nyong, the third wife of Kapitan Chung Keng Quee."

In [14]:
t, ids = normalizer.normalize(text)
t

"george town , a heritage activist has questioned the penang government ' s continued silence on whether it would severely punish a land owner for illegally demolishing the - year - old grave of foo teng nyong , the third wife of kapitan chung keng quee ."

In [15]:
ids = intersperse(ids, 0)

In [16]:
ids = torch.LongTensor(ids)
ids_lengths = torch.LongTensor([ids.size(0)])
ids = ids.unsqueeze(0)

In [17]:
audio = model.infer(
    ids,
    ids_lengths,
    noise_scale=0.6666,
    noise_scale_w=0.6666,
    length_scale=1.0,
)

In [18]:
audio = audio[0][0,0].detach().numpy()

In [19]:
import IPython.display as ipd
ipd.Audio(audio, rate = 22050)

In [20]:
torch.save(model.state_dict(), 'female-singlish.pth')

In [21]:
from malaya_boilerplate.huggingface import upload_dict

In [22]:
files_mapping = {'female-singlish.pth': 'model.pth',
                config: 'config.json'}
upload_dict(model = 'VITS-female-singlish', files_mapping = files_mapping, username = 'mesolitica')

/home/ubuntu/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:79: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.7. Pass `repo_id` instead.
  warnings.warn(
409 Client Error: Conflict for url: https://huggingface.co/api/repos/create - You already created this model repo
